# Dependencies and Imports

In [1]:
#@title Install dependencies

!pip install -q torchaudio omegaconf

import torch
from pprint import pprint
from omegaconf import OmegaConf
from IPython.display import Audio, display

torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)
models = OmegaConf.load('latest_silero_models.yml')

     |████████████████████████████████| 1.9MB 7.8MB/s 
     |████████████████████████████████| 645kB 39.5MB/s 


# Demo

In [ ]:
# see latest avaiable models
available_languages = list(models.tts_models.keys())
print(f'Available languages {available_languages}')

for lang in available_languages:
    speakers = list(models.tts_models.get(lang).keys())
    print(f'Available speakers for {lang}: {speakers}')

In [2]:
import torch

language = 'ru'
speaker = 'baya_16khz'
device = torch.device('cpu')
model, symbols, sample_rate, example_text, apply_tts = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                                                      model='silero_tts',
                                                                      language=language,
                                                                      speaker=speaker)
model = model.to(device)  # gpu or cpu

Downloading: "https://github.com/snakers4/silero-models/archive/master.zip" to /root/.cache/torch/hub/master.zip


In [8]:
audio = apply_tts(texts=["В н+едрах т+ундры в+ыдры в г+етрах т+ырят в в+ёдра +ядра к+едров."],
                  model=model,
                  sample_rate=sample_rate,
                  symbols=symbols,
                  device=device)

print(example_text)
display(Audio(audio[0], rate=sample_rate))

В н+едрах т+ундры в+ыдры в г+етрах т+ырят в в+ёдра +ядра к+едров.


## Enhance synthesis with logmmse

In [9]:
!pip install -q logmmse

You can try to enhance synthesized audio with logmmse algorithm, though it could demand parameters tuning for the particular speaker.

In [10]:
import numpy as np
from logmmse import logmmse

enhanced = logmmse(np.array(audio[0]), sample_rate, output_file=None, initial_noise=1, window_size=160, noise_threshold=0.15)
display(Audio(enhanced, rate=sample_rate))